N2 -> CO APDFT

In [1]:
from pyscf import gto,scf
import pyscf
import numpy as np

pyscf.__version__

'2.2.1'

In [2]:
from AP_class import APDFT_perturbator as AP

In [3]:
mol_NN=gto.M(atom= "N 0 0 0; N 0 0 2.1",unit="Bohr",basis="def2-TZVP")
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = 'pbe0'
# mf_nn.xc = 'lda,vwn'
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 3
e = mf_nn.scf(dm0=mf_nn.init_guess_by_1e())
print(mf_nn.e_tot)
print(mf_nn.energy_nuc())
mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1])
print(mf_nn.mol.atom_coords())

converged SCF energy = -108.984873786865
-108.98487378686538
23.333333333333332
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000000000    -0.1456458004
1 N     0.0000000000     0.0000000000     0.1456458004
----------------------------------------------
[[0.  0.  0. ]
 [0.  0.  2.1]]


Calculate $\partial E/\partial Z_i$, hessian $\partial^2E/\partial Z_i\partial Z_j$ and cubic derivatives $\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [4]:
ap_nn.build_elec_gradient()
ap_nn.build_elec_hessian()
ap_nn.build_cubic()

array([[[-0.21518468,  0.11800518],
        [ 0.11800518,  0.11800518]],

       [[ 0.11800518,  0.11800518],
        [ 0.11800518, -0.21518468]]])

CO

In [5]:
print(ap_nn.elec_APDFT0(np.asarray([-1,1])))
print(ap_nn.elec_APDFT1(np.asarray([-1,1])))
print(ap_nn.elec_APDFT2(np.asarray([-1,1])))
print(ap_nn.elec_APDFT3(np.asarray([-1,1])))

-132.3182071201987
-132.31820712019987
-135.5648258581067
-135.5648258581067


In [6]:
print(ap_nn.elec_APDFT0(np.asarray([1,-1])))
print(ap_nn.elec_APDFT1(np.asarray([1,-1])))
print(ap_nn.elec_APDFT2(np.asarray([1,-1])))
print(ap_nn.elec_APDFT3(np.asarray([1,-1])))

-132.3182071201987
-132.31820712019754
-135.56482585810437
-135.56482585810437


In [7]:
mol_CO=gto.M(atom= "C 0 0 0; O 0 0 2.1",unit="Bohr",basis="def2-TZVP")
mf_co=scf.RHF(mol_CO)
# mf_co.xc = 'pbe0'
e = mf_co.scf(dm0=mf_co.init_guess_by_1e())
print(e)
print(mf_co.energy_nuc())
print(mf_co.e_tot - mf_co.energy_nuc())

converged SCF energy = -112.786968394493
-112.78696839449282
22.857142857142854
-135.64411125163568


In [8]:
from utils import calc_nuclei_repulsion_energy

charges = [6, 8]
coords = np.asarray([[0, 0, 0], [0, 0, 2.1]])
nuc_repul_ene = calc_nuclei_repulsion_energy(charges, coords, unit_angstrom=False)
print(nuc_repul_ene)

22.857142857142858


In [9]:
mol_NN_sto=gto.M(atom= "N 0 0 0; N 0 0 2.1",unit="Bohr",basis="sto-3g")
mf_nn_sto=scf.RHF(mol_NN_sto)
# Note that the use of dm0=mf_nn_sto.init_guess_by_1e() gives higher energy
# e = mf_nn_sto.scf(dm0=mf_nn_sto.init_guess_by_1e())
e = mf_nn_sto.scf()
print(mf_nn_sto.e_tot)
print(mf_nn_sto.energy_nuc())
mf_nn_sto.nuc_grad_method().kernel()
ap_nn_sto=AP(mf_nn_sto,sites=[0,1])
ap_nn_sto.build_elec_gradient()
ap_nn_sto.build_elec_hessian()
ap_nn_sto.build_cubic()

print(ap_nn_sto.elec_APDFT0(np.asarray([-1,1])) + nuc_repul_ene)
print(ap_nn_sto.elec_APDFT1(np.asarray([-1,1])) + nuc_repul_ene)
print(ap_nn_sto.elec_APDFT2(np.asarray([-1,1])) + nuc_repul_ene)
print(ap_nn_sto.elec_APDFT3(np.asarray([-1,1])) + nuc_repul_ene)

converged SCF energy = -107.49885049543
-107.49885049543037
23.333333333333332
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000000000     0.0865931477
1 N     0.0000000000     0.0000000000    -0.0865931477
----------------------------------------------
-107.97504097162084
-107.97504097162081
-109.0133279713045
-109.0133279713045


In [10]:
charges = [7, 7]
charges = [6, 7]
nuc_repul_ene = calc_nuclei_repulsion_energy(charges, coords, unit_angstrom=False)
print(ap_nn_sto.elec_APDFT1(np.asarray([-1,0])) + nuc_repul_ene)
charges = [7, 8]
nuc_repul_ene = calc_nuclei_repulsion_energy(charges, coords, unit_angstrom=False)
print(ap_nn_sto.elec_APDFT3(np.asarray([0,1])) + nuc_repul_ene)
charges = [8, 8]
nuc_repul_ene = calc_nuclei_repulsion_energy(charges, coords, unit_angstrom=False)
print(ap_nn_sto.elec_APDFT3(np.asarray([1,1])) + nuc_repul_ene)

-89.53632873812917
-125.73429803367412
-143.0149071258722


Use of alchemical_calculator class

In [11]:
from alch_calc import alchemical_calculator as ac

target_mol = "N 0 0 0; N 0 0 2.1"
dft_functional = None
name_basis_set = "sto-3g"
nuc_charges_vec = [-1, 1]

ac_mol = ac(target_mol, name_basis_set, dft_functional, sites=[0,1], coord_unit="Bohr")
ac_mol.calc_all_derivatives()

converged SCF energy = -107.498850545116


In [12]:
charges = [6, 8]
coords = np.asarray([[0, 0, 0], [0, 0, 2.1]])
nuc_repul_ene = calc_nuclei_repulsion_energy(charges, coords, unit_angstrom=False)

In [13]:
print(ac_mol.get_elec_APDFT0(nuc_charges_vec) + nuc_repul_ene)
print(ac_mol.get_elec_APDFT1(nuc_charges_vec) + nuc_repul_ene)
print(ac_mol.get_elec_APDFT2(nuc_charges_vec) + nuc_repul_ene)
print(ac_mol.get_elec_APDFT3(nuc_charges_vec) + nuc_repul_ene)

-107.9750410213068
-107.9750410213068
-109.0133280597324
-109.0133280597324
